# Intro Notes

This notebook takes the two CSV files which were either provided or generated with the "data-crunching" notebook, and visualizes it in an interactive choropleth map.
Please make sure that the file is located in the same folder as this notebook. Once you've verified this, make sure that the file name within the quotation marks below matches the filename, including the ".csv" extension. Then, run the whole notebook. The notebook will generate an interactive in line with the code; in some cases, the notebook may generate a link to the interactive instead. 

In [1]:
import pandas as pd
from urllib.request import urlopen
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import json

In [2]:
df = pd.read_csv("priority_statistic_df.csv", converters = {"GEOID":str})

#These lists help define what risks are available within the data to visualize. 
names = ["avalache_score", "coastal_flooding_score", "cold_wave_score", "drought_score", "earthquake_score", "hail_score", "heat_wave_score", "hurricane_score", 
         "ice_storm_score", "landslide_score", "lightning_score", "riverine_flooding_score", "strong_wind_score", "tornado_score", "tsunami_score", "volcano_score", 
         "wildfire_score", "winter_weather_score"]
labels = ["Avalache", "Coastal Flooding", "Cold Wave", "Drought", "Earthquake", "Hail", "Heat Wave", "Hurricane", 
         "Ice Storm", "Landslide", "Lightning", "Riverine Flooding", "Strong Wind", "Tornado", "Tsunami", "Volcano", 
         "Wildfire", "Winter Weather"]
dict_names = dict(zip(names, labels))

# Dash Interactive

In [ ]:
app = Dash(__name__)


app.layout = html.Div([
html.H2("Undermapped and At Risk Analysis"),
html.Div([
    html.H4("Select a natural disaster:"),
    dcc.Dropdown(
        id="disaster", 
        options=
        dict_names,
        #inline=True,
        value="hurricane_score"
    )],style={"width": "25%"}),
    html.Div([
    html.H4("Select your desired ratio:"),
    html.P("Selecting 0 will visualize only undermapped data, selecting 100 will visualize only risk data."),
    dcc.Slider(min = 0, max = 100, step = 25, 
           value=75,
           id="ratio"
    )]),

    dcc.Graph(id="graph", responsive = "auto"),
],style = {"font-family":    ["Arial", "Helvetica", "sans-serif"]})

with urlopen("https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json") as response:
    counties = json.load(response)
    
@app.callback(
    Output("graph", "figure"), 
    Input("disaster", "value"),
    Input("ratio", "value"))

def display_choropleth(disaster, ratio):
    geojson = counties
    filtered_df = df[df.ratio == ratio]
    fig = px.choropleth(filtered_df, geojson=counties, locations="GEOID", color=disaster,
                    color_continuous_scale="Bluyl",
                    range_color=(0, 100),
                    scope="usa",
                    labels={disaster:"Combined Risk Score", "GEOID":"Fips Code", "COUNTY":"County","STATE":"State"},
                    hover_data=["NAMELSAD","STATE_NAME"]
                          )
    #fig.update_geos(fitbounds="locations", visible=True, center={"lon":50, "lat":-95})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig


app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
